In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import os
import random
from PIL import Image, ImageTk
import datetime

# ------------------ 配置区 ------------------
DATA_FILE = "xxxxxxx"       # 打分记录文件
IMAGES_FOLDER = "xxxxxx"  # 图片文件夹
FINAL_SCORES_FILE = "xxxxxx"  # 多次次打分满后自动记录的最终平均分(可有可无)

# ------------------ 加载或初始化数据 ------------------
if not os.path.exists(DATA_FILE):
    data = pd.DataFrame(columns=["Name", "Age", "Gender", "Image ID", "Time",
                                 "人性化尺度", "安全性", "复杂性", "封闭性", "意向性", "自然舒适性", "步行性"])# 此为作者设置指标，请根据个人研究方向修改
else:
    data = pd.read_csv(DATA_FILE)
if not os.path.exists(FINAL_SCORES_FILE):
    final_scores = pd.DataFrame(columns=["Image ID", "人性化尺度", "安全性", "复杂性", "封闭性",
                                         "意向性", "自然舒适性", "步行性"])# 此为作者设置指标，请根据个人研究方向修改
else:
    final_scores = pd.read_csv(FINAL_SCORES_FILE)

# 读取图片文件夹中的所有图片文件
image_files = [f for f in os.listdir(IMAGES_FOLDER)
               if f.endswith(('.png', '.jpg', '.jpeg'))]

current_rater = None  # 当前评分者信息
rated_images_by_current_rater = set()  # 当前评分者已评分的图片集合

# ------------------ 函数区 ------------------
def save_data():
    """保存评分数据到CSV文件"""
    data.to_csv(DATA_FILE, index=False)

def save_final_scores():
    """保存最终得分数据到CSV文件"""
    final_scores.to_csv(FINAL_SCORES_FILE, index=False)

def load_image(image_id):
    """加载并调整图片大小以适应显示区域"""
    image_path = os.path.join(IMAGES_FOLDER, image_id)
    image = Image.open(image_path)
    screen_width = root.winfo_screenwidth()
    image.thumbnail((screen_width, screen_width * 0.6), Image.Resampling.LANCZOS)
    return ImageTk.PhotoImage(image)

def update_progress_labels():
    """更新界面上的已评分和未评分图片数量显示"""
    rated_count = len(rated_images_by_current_rater)
    remaining_count = 30 - rated_count  # 当前每人打分上限为 30 张，请根据个人研究方向修改
    label_rated_count.config(text=f"已评价图片数量：{rated_count}")
    label_remaining_count.config(text=f"剩余评价图片数量：{remaining_count}")

def add_score():
    """添加评分记录，并更新数据和界面"""
    global data, current_rater, rated_images_by_current_rater, image_files

    # 若已达 30 张，不再评分，请根据个人研究方向修改数量
    if len(rated_images_by_current_rater) >= 30:
        messagebox.showinfo("信息", "已达评分图片上限！")
        return

    image_id = current_image_id.get()
    scores = [int(scale.get()) for scale in scales]
    time_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # 构造新行
    new_row = [
        current_rater["Name"], 
        current_rater["Age"], 
        current_rater["Gender"],
        image_id, 
        time_now
    ] + scores

    # 列数检查
    if len(new_row) != len(data.columns):
        messagebox.showerror("错误", "列数不匹配!")
        return

    data = pd.concat([data, pd.DataFrame([new_row], columns=data.columns)], ignore_index=True)
    rated_images_by_current_rater.add(image_id)

    save_data()
    refresh_table()

    # 判断同一张图片 >=5 次，此图片就移除，请根据个人研究方向修改
    if len(data[data["Image ID"] == image_id]) >= 5:
        if image_id in image_files:
            image_files.remove(image_id)
        compute_final_score_for_image(image_id)

    show_next_image()
    update_progress_labels()

def compute_final_score_for_image(image_id):
    global final_scores

    if image_id in final_scores["Image ID"].values:
        return

    sub_df = data[data["Image ID"] == image_id]
    if len(sub_df) < 5:
        return  # 不足5次，不计算
    
    cols = ["人性化尺度", "安全性", "复杂性", "封闭性", "意向性", "自然舒适性", "步行性"]
    final_values = []
    for c in cols:
        scores_for_col = sorted(list(sub_df[c]))
        mid_scores = scores_for_col[1:-1]
        if len(mid_scores) == 3:
            avg_val = sum(mid_scores) / 3
        else:
            avg_val = sum(mid_scores) / max(len(mid_scores), 1)
        final_values.append(round(avg_val, 2))

    new_final_row = [image_id] + final_values
    df_temp = pd.DataFrame([new_final_row], columns=["Image ID"] + cols)
    final_scores = pd.concat([final_scores, df_temp], ignore_index=True)
    save_final_scores()

def show_next_image():
    """显示下一张待评分的图片"""
    global image_files, rated_images_by_current_rater

    # 查找评分次数未满5次，请根据个人研究方向修改
    available_images = [
        img for img in image_files
        if img not in rated_images_by_current_rater
        and len(data[data["Image ID"] == img]) < 5
    ]

    if available_images:
        next_image = random.choice(available_images)
        img_obj = load_image(next_image)
        label_image.config(image=img_obj)
        label_image.image = img_obj
        current_image_id.set(next_image)
        reset_scales()
    else:
        messagebox.showinfo("信息", "所有图片已被评分或无可用图片！")
        root.destroy()

def reset_scales():
    for scale, label in zip(scales, scale_labels):
        scale.set(5)
        update_scale_label(scale, label)

def refresh_table():
    for row in tree.get_children():
        tree.delete(row)
    sorted_data = data.sort_values("Time", ascending=False)
    for index, row_data in sorted_data.iterrows():
        tree.insert("", "end", values=[
            row_data["Name"],
            row_data["Image ID"],
            len(data[data["Image ID"] == row_data["Image ID"]]),
            row_data["Time"]
        ])

def show_rater_popup():#请根据个人研究方向修改打分者信息填写项
    popup = tk.Toplevel(root)
    popup.title("评分者信息")
    popup.geometry("300x300")

    tk.Label(popup, text="评分者姓名:").pack(pady=5)
    entry_name = tk.Entry(popup)
    entry_name.pack(pady=5)

    tk.Label(popup, text="评分者性别:").pack(pady=5)
    entry_gender = tk.Entry(popup)
    entry_gender.pack(pady=5)

    tk.Label(popup, text="评分者年龄:").pack(pady=5)
    entry_age = tk.Entry(popup)
    entry_age.pack(pady=5)

    tk.Button(
        popup,
        text="确定",
        command=lambda: save_rater_info(entry_name.get(), entry_gender.get(), entry_age.get(), popup)
    ).pack(pady=10)

def save_rater_info(name, gender, age, popup):
    global current_rater
    current_rater = {"Name": name, "Gender": gender, "Age": age}
    popup.destroy()
    root.deiconify()
    show_next_image()

def update_scale_label(scale, label):
    value = scale.get()
    if value < 2:
        label.config(text="很差")
    elif value < 4:
        label.config(text="较差")
    elif value < 6:
        label.config(text="一般")
    elif value < 8:
        label.config(text="还行")
    elif value < 10:
        label.config(text="很不错")
    else:
        label.config(text="非常好！")

# ------------------ 主程序界面 ------------------
root = tk.Tk()
root.title("图片评分工具")
root.state('zoomed')
root.withdraw()

top_frame = tk.Frame(root)
top_frame.pack(side="top", fill="x")
top_label = tk.Label(top_frame, text="", font=("Helvetica", 12))
top_label.pack()

image_frame = tk.Frame(root, pady=0)
image_frame.pack(side="top", fill="both")

label_image = tk.Label(image_frame)
label_image.pack()

current_image_id = tk.StringVar()

progress_frame = tk.Frame(root, pady=0)
progress_frame.pack(side="top", fill="x")
label_rated_count = tk.Label(progress_frame, text="已评价图片数量：0", font=("Helvetica", 12))
label_rated_count.pack(side="left", padx=4)
label_remaining_count = tk.Label(progress_frame, text="剩余评价图片数量：30", font=("Helvetica", 12))
label_remaining_count.pack(side="right", padx=4)

bottom_frame = tk.Frame(root)
bottom_frame.pack(side="bottom", fill="both", expand=True)

table_frame = tk.Frame(bottom_frame)
table_frame.pack(side="left", fill="both", expand=True)
columns = ["评分者姓名", "图片编号", "已打分次数", "评分时间"]
tree = ttk.Treeview(table_frame, columns=columns, show="headings")
tree.pack(side="left", fill="both", expand=True)

for col in columns:
    tree.heading(col, text=col)
    tree.column(col, anchor="center", width=100)

scrollbar = tk.Scrollbar(table_frame, orient="vertical", command=tree.yview)
scrollbar.pack(side="right", fill="y")
tree.configure(yscrollcommand=scrollbar.set)

scale_frame_outer = tk.Frame(bottom_frame, padx=20)
scale_frame_outer.pack(side="right", fill="both", expand=True)

scrollbar = ttk.Scrollbar(scale_frame_outer, orient="vertical")
canvas = tk.Canvas(scale_frame_outer, yscrollcommand=scrollbar.set)
scrollbar.config(command=canvas.yview)
scrollbar.pack(side="right", fill="y")
canvas.pack(side="left", fill="both", expand=True)
scrollable_frame = ttk.Frame(canvas)

# 鼠标滚轮支持(多操作系统)
def _on_mousewheel_windows(event):
    canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")

def _on_mousewheel_mac(event):
    if event.num == 5 or event.delta == -120:
        canvas.yview_scroll(1, "units")
    elif event.num == 4 or event.delta == 120:
        canvas.yview_scroll(-1, "units")

canvas.bind_all("<MouseWheel>", _on_mousewheel_windows)
canvas.bind_all("<Button-4>", _on_mousewheel_mac)
canvas.bind_all("<Button-5>", _on_mousewheel_mac)

canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
scrollable_frame.bind("<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all")))

labels = ["人性化尺度", "安全感", "复杂性", "围合度", "独特性", "绿化", "可步行性"]
scales = []
scale_labels = []
for i, label_text in enumerate(labels):
    tk.Label(scrollable_frame, text=label_text, font=("Helvetica", 12)).pack(fill="x")
    scale = tk.Scale(scrollable_frame, from_=0, to=10, orient="horizontal", resolution=1, length=600, width=20)
    scale.set(5)
    scale.pack(fill="x", expand=True)
    scales.append(scale)

    scale_label = tk.Label(scrollable_frame, text="一般", font=("Helvetica", 10))
    scale_label.pack()
    scale_labels.append(scale_label)

    scale.configure(command=lambda value, s=scale, l=scale_label: update_scale_label(s, l))

btn_submit = tk.Button(scrollable_frame, text="提交评分", command=add_score)
btn_submit.pack(pady=20)

show_rater_popup()
root.mainloop()

: 